In [1]:
feature_name = [
    "acc_cost_usd",
    "acc_first_purchase_price_usd",
    "acc_role_cnt",
    "acc_total_charge_cnt",
    "acc_total_charge_sum_usd",
    "acc_total_refund_cnt",
    "acc_total_refund_sum_usd",
    "bundle_pop_buy_count_last_day",
    "bundle_pop_buy_count_last_day_campaign_stage_trigger",
    "bundle_pop_buy_count_last_day_tech_unlock_trigger",
    "bundle_pop_buy_count_last_week",
    "bundle_pop_buy_count_last_week_adjust",
    "bundle_pop_buy_max_last_day",
    "bundle_pop_buy_max_last_week",
    "bundle_pop_buy_max_last_week_adjust",
    "bundle_pop_buy_max_price_campaign_stage_trigger",
    "bundle_pop_buy_max_price_campaign_stage_trigger_commax",
    "bundle_pop_buy_max_price_tech_unlock_trigger",
    "bundle_pop_buy_max_price_tech_unlock_trigger_commax",
    "bundle_pop_buy_sum_last_day",
    "bundle_pop_buy_sum_last_week",
    "bundle_pop_buy_sum_last_week_adjust",
    "bundle_pop_count_campaign_stage_trigger_9999",
    "bundle_pop_count_last_day",
    "bundle_pop_count_last_day_campaign_stage_trigger",
    "bundle_pop_count_last_day_tech_unlock_trigger",
    "bundle_pop_count_last_week",
    "bundle_pop_count_last_week_adjust",
    "bundle_pop_count_tech_unlock_trigger_9999",
    "buy_rate_last_day",
    "buy_rate_last_day_campaign_stage_trigger",
    "buy_rate_last_day_tech_unlock_trigger",
    "buy_rate_last_week",
    "if_accounts_first_role",
    "if_pruchase_active_in_1day",
    "if_pruchase_active_in_30day",
    "if_pruchase_active_in_3day",
    "if_pruchase_active_in_7day",
    "is_acc_transfer_from_old",
    "is_acc_transfer_to_new",
    "is_role_transfer_from_old",
    "is_role_transfer_to_new",
    "is_valid",
    "level",
    "max_role_level",
    "money_flow_in_cout_last_day",
    "money_flow_in_cout_last_week",
    "money_flow_in_cout_last_week_adjust",
    "money_flow_out_cout_last_day",
    "money_flow_out_cout_last_week",
    "money_flow_out_cout_last_week_adjust",
    "purchase_price_1499_last_week",
    "purchase_price_1499_last_week_adjust",
    "purchase_price_1999_last_week",
    "purchase_price_1999_last_week_adjust",
    "purchase_price_2999_last_week",
    "purchase_price_2999_last_week_adjust",
    "purchase_price_4999_last_week",
    "purchase_price_4999_last_week_adjust",
    "purchase_price_499_last_week",
    "purchase_price_499_last_week_adjust",
    "purchase_price_9999_last_week",
    "purchase_price_9999_last_week_adjust",
    "purchase_price_999_last_week",
    "purchase_price_999_last_week_adjust",
    "purchase_price_max",
    "purchase_price_max_last_3days",
    "purchase_price_max_last_day",
    "purchase_price_max_last_month",
    "purchase_price_max_last_week",
    "purchase_price_max_last_week_adjust",
    "purchase_price_sum_last_3days",
    "purchase_price_sum_last_day",
    "purchase_price_sum_last_month",
    "purchase_price_sum_last_week",
    "purchase_price_sum_last_week_adjust",
    "purchase_price_sum_max_ratio",
    "purchase_price_sum_max_ratio_com2",
    "role_cost_usd",
    "role_create_os_android",
    "role_create_os_ios",
    "role_first_purchase_price_usd",
    "role_last_login_duration",
    "role_last_purchase_price_usd",
    "role_lifespan",
    "role_login_last_now",
    "role_login_secondlast_last",
    "role_login_secondlast_now",
    "role_logout_last_now",
    "role_no",
    "role_purchase_first_last",
    "role_purchase_first_now",
    "role_purchase_last_now",
    "role_total_charge_cnt",
    "role_total_charge_sum_usd",
    "role_total_refund_cnt",
    "role_total_refund_sum_usd",
    "role_type_fraud",
    "role_type_no_install_info",
    "vip_level",
]

s = "column_name : {}\n\n".format(",".join(feature_name))
for slot, x in enumerate(feature_name):
    s += f"feature_name=r_{x};depend={x};method=Float;slot={slot};args=-100\n"

with open("../../config/dynamic_pricing/0.0.2.conf", "r+") as f:
    f.write(s)


In [3]:
import sys
sys.path.append("../../")

from cityhash import CityHash32
import pandas as pd
import numpy as np
from scipy.stats import boxcox

from config.HIVESERVER2_token import *
from lmtk.feature_extraction import (
    FEATURE_BIT,
    FEATURE_KEY_SLOT_PREFIX,
    FeatureExtractor,
)
from lmtk.feature_extraction.constants import *
from pyutil.pysql.sql import SQLClient

fe = FeatureExtractor('../../config/dynamic_pricing/0.0.2.conf')
client = SQLClient('mix')

client.execute_trino_query("select * from ml_tmp_db.training_sample_for_wgame_bundle_price_002 where grp = 'maxp7day'")
data = client.to_pandas()

data.to_parquet('../../data/raw/0.0.2.maxp7day.pqt', compression='gzip')
# data  = pd.read_parquet('../../data/raw/0.0.2.maxp7day.pqt').query("  dt <= '2023-04-26' ")
# data.shape[0]

In [5]:
data = data.assign(
    feature = lambda x: x.feature.apply(fe.extract),
    bundle_price = lambda x: x.bundle_price.astype(np.int64),
    sid = (data['acc_id'] + data['role_id'] + data['bundle_price'].astype(str)).apply(CityHash32),
)

data = data.assign(
    ** {
        x : data.feature.apply(lambda y: y.get(f'r_{x}')[0])
        for x in feature_name
    }
).filter(items = feature_name + ['trigger_type','bundle_price','if_buy_pop_gift','sid'])


cat_binary = []
for x in feature_name:
    if data[x].value_counts().shape[0] <= 2:
        cat_binary.append(x)


dense_feature = [x for x in feature_name if x not in cat_binary]
boxcox_vocab = dict()
logbucket_vocab = dict()

for x in dense_feature:
    c = data[x].min()
    c = abs(c) + 1
    _, l = boxcox(data[x] + c, None, None) 
    boxcox_vocab[x] = (l,c)

    v = data[x] + c
    minv, maxv = v.min(), v.max()

    logbucket_vocab[x] = np.exp(np.log(maxv)/20)

C:\Users\caoxu\AppData\Local\Temp\ipykernel_16224\4246301852.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data = data.assign(
C:\Users\caoxu\AppData\Local\Temp\ipykernel_16224\4246301852.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data = data.assign(
d:\Anaconda\envs\tf210\lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
d:\Anaconda\envs\tf210\lib\site-packages\numpy\core\_methods.py:247: RuntimeWarning: overflow encountered in reduce
  r

In [6]:
import tensorflow as tf
import tfutils

names = feature_name+ ['trigger_type','bundle_price','if_buy_pop_gift']
price_chain = data.bundle_price.value_counts().sort_index().index.to_list()
idx2p = dict(enumerate(price_chain))
p2idx = {v:k for k,v in idx2p.items()}

def getdataset(query = 'sid % 10<=7'):
    def get_label_func(x):
        price = x.pop('bundle_price')
        price = tf.one_hot(price, len(price_chain))
        label = x.pop('if_buy_pop_gift')
        return x, price
    
    tensors = tuple()
    data1 = data.query(query)
    for f in names:
        if f == 'bundle_price':
            v = data1[f].apply(p2idx.get).values
        else:
            v = data1[f].values
        t = tf.data.Dataset.from_tensor_slices(v, name = f)
        tensors = tensors + (t,)
        
    return tf.data.Dataset.zip(tensors).map(lambda *x: dict(zip(names, x))).map(get_label_func)

trainset = getdataset('sid % 10<=7')
valset = getdataset('sid % 10>7')

In [7]:
data.bundle_price.value_counts()

499     122657
999     107799
1999     99135
9999     92894
4999     24339
1499     15004
2999     10397
99        9274
Name: bundle_price, dtype: int64

In [8]:
class GatedLinearUnit(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.linear = tf.keras.layers.Dense(units)
        self.sigmoid = tf.keras.layers.Dense(units, activation="sigmoid")

    def call(self, inputs):
        return self.linear(inputs) * self.sigmoid(inputs)

class GatedResidualNetwork(tf.keras.layers.Layer):
    def __init__(self, units, dropout_rate):
        super().__init__()
        self.units = units
        self.elu_dense = tf.keras.layers.Dense(units, activation="elu")
        self.linear_dense = tf.keras.layers.Dense(units)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.gated_linear_unit = GatedLinearUnit(units)
        self.layer_norm = tf.keras.layers.BatchNormalization()
        self.project = tf.keras.layers.Dense(units)

    def call(self, inputs):
        x = self.elu_dense(inputs)
        x = self.linear_dense(x)
        x = self.dropout(x)
        if inputs.shape[-1] != self.units:
            inputs = self.project(inputs)
        x = inputs + self.gated_linear_unit(x)
        x = self.layer_norm(x)
        return x

class VariableSelection(tf.keras.layers.Layer):
    def __init__(self, num_features, units, dropout_rate):
        super().__init__()
        self.grns = list()
        # Create a GRN for each feature independently
        for idx in range(num_features):
            grn = GatedResidualNetwork(units, dropout_rate)
            self.grns.append(grn)
        # Create a GRN for the concatenation of all the features
        self.grn_concat = GatedResidualNetwork(units, dropout_rate)
        self.softmax = tf.keras.layers.Dense(units=num_features, activation="softmax")

    def call(self, inputs):
        v = tf.keras.layers.concatenate(inputs)
        v = self.grn_concat(v)
        v = tf.expand_dims(self.softmax(v), axis=-1)

        x = []
        for idx, input in enumerate(inputs):
            x.append(self.grns[idx](input))
        x = tf.stack(x, axis=1)

        outputs = tf.squeeze(tf.matmul(v, x, transpose_a=True), axis=1)
        return outputs

class FieldVariableSelection(tf.keras.layers.Layer):
    def __init__(self, num_features, units, dropout_rate):
        super().__init__()
        self.grn = GatedResidualNetwork(units, dropout_rate)
        self.grn_concat = GatedResidualNetwork(units, dropout_rate)
        self.softmax = tf.keras.layers.Dense(units=num_features, activation="softmax")

    def call(self, inputs):
        v = tf.keras.layers.Flatten()(inputs)
        v = self.grn_concat(v)
        v = tf.expand_dims(self.softmax(v), axis=-1)
        x = self.grn(inputs)
        outputs = tf.squeeze(tf.matmul(v, x, transpose_a=True), axis=1)
        return outputs
    
class VariableSelection(tf.keras.layers.Layer):
    def __init__(self, num_features, units, dropout_rate):
        super().__init__()
        self.grns = list()
        # Create a GRN for each feature independently
        for idx in range(num_features):
            grn = GatedResidualNetwork(units, dropout_rate)
            self.grns.append(grn)
        # Create a GRN for the concatenation of all the features
        self.grn_concat = GatedResidualNetwork(units, dropout_rate)
        self.softmax = tf.keras.layers.Dense(units=num_features, activation="softmax")

    def call(self, inputs):
        v = tf.keras.layers.concatenate(inputs)
        v = self.grn_concat(v)
        v = tf.expand_dims(self.softmax(v), axis=-1)

        x = []
        for idx, input in enumerate(inputs):
            x.append(self.grns[idx](input))
        x = tf.stack(x, axis=1)

        outputs = tf.squeeze(tf.matmul(v, x, transpose_a=True), axis=1)
        return outputs

class LogBucketLayer(tf.keras.layers.Layer):
    def __init__(self, bins = 10, c = 1.0):
        super().__init__()
        self.bins = bins
        self.c = c
        self.initializer = tf.keras.initializers.RandomUniform(minval = 1.5, maxval = np.e ** 2)

    def build(self, input_shape):
        super().build(input_shape)
        self.idx = self.add_weight(
            shape=(1,1),
            initializer=self.initializer,
            name = 'log_bucket'
        )
    

    @tf.function
    def call(self, x):
        x = tf.math.divide_no_nan(tf.math.log(x + self.c), tf.math.log(self.idx))
        x = tf.math.floor(x)
        x = tf.clip_by_value(x, 0, self.bins - 1)
        x = tf.cast(x, tf.int32)
        return x

In [9]:
embedding_size = 16

inputs = []
embeddings = []

for x in dense_feature:
    i = tf.keras.Input(shape = (1,), dtype=tf.float32, name = x)
    l = tf.constant(boxcox_vocab[x][0], dtype=tf.float32)
    c = tf.constant(boxcox_vocab[x][1], dtype=tf.float32)
    b = tf.constant(logbucket_vocab[x], dtype=tf.float32)

    i1 = (tf.math.pow(i+c, l) - 1)/l
    e1 = tf.keras.layers.Dense(embedding_size, activation='leaky_relu')(i)
    e1 = tf.expand_dims(e1, axis = 1)

    i2 = tf.math.log(i+c) / tf.math.log(b)
    i2 = tf.clip_by_value(i2, 0, 19)
    e2 = tf.keras.layers.Embedding(20, embedding_size, activity_regularizer= tf.keras.regularizers.L2(1e-4))(i2)

    e = tf.concat([e1, e2], axis = -1)
    
    inputs.append(i)
    embeddings.append(e)

for x in cat_binary:
    i = tf.keras.Input(shape = (1,), dtype=tf.float32, name = x)
    e = tf.cast(i, tf.int32)
    e = tf.clip_by_value(e, 0, 1)
    e = tf.keras.layers.Embedding(2, embedding_size * 2,activity_regularizer= tf.keras.regularizers.L2(1e-4))(e)
    inputs.append(i)
    embeddings.append(e)

for x in ['trigger_type']:
    i = tf.keras.Input(shape = (1,), dtype=tf.string, name = x)
    vocab = "account_lvl_trigger,mainbase_lvl_trigger,campaign_stage_trigger,tech_unlock_trigger,building_trigger-10001,building_trigger-10002,building_trigger-10003,building_trigger-10004,building_trigger-10005,building_trigger-10006,building_trigger-10014,building_trigger-10015,building_trigger-10018,building_trigger-10010,building_trigger-10011,building_trigger-10012,building_trigger-10013,building_trigger-10024,building_trigger-10027,building_trigger-10028,building_trigger-10029,building_trigger-10030,card_unlock_trigger_ground_force,card_unlock_trigger_air_force,gift_login,gvg_trigger,teleport_trigger,population_consume_trigger,blueprint_max_trigger_ground_force,blueprint_max_trigger_air_force,advancecard_unlock_trigger_ground_force,advancecard_unlock_trigger_air_force,mechanician_trigger,tech_lv_trigger,multi_tech_lv_trigger,blackmall_trigger,all_officer_lv_trigger,officer_lv_trigger,drawing_trigger,replacement_trigger,replacement_recycle_trigger,card_unlock_trigger,building_trigger-10020"
    vocab = vocab.split(',')
    e = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token=0)(i)
    e = tf.keras.layers.Embedding(input_dim = len(vocab), output_dim = embedding_size * 2, mask_zero = True, activity_regularizer= tf.keras.regularizers.L2(1e-4))(e)

    inputs.append(i)
    embeddings.append(e) 

x = tf.concat(embeddings, axis = 1)
res = FieldVariableSelection(len(embeddings),32,0.1)(x)

x = tfutils.layers.attn.BatchNormTransformerEncoder(32, 8, 32)(x)
x = tfutils.layers.attn.BatchNormTransformerEncoder(32, 4, 32)(x)
x = tfutils.layers.attn.BatchNormTransformerEncoder(32, 4, 32)(x)

x = FieldVariableSelection(len(embeddings),32,0.1)(x)


x = x + res
x = tf.keras.layers.Dense(len(p2idx), activation='softmax')(x)

model = tf.keras.Model(inputs = inputs, outputs = x)
sum([tf.keras.backend.count_params(p) for p in model.trainable_variables])

d:\Anaconda\envs\tf210\lib\site-packages\numpy\core\numeric.py:2463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


540210

In [10]:
lr = tf.keras.optimizers.schedules.CosineDecay(1e-3,decay_steps = 500,alpha = 0.1)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
    # optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=lr),
    loss=tf.keras.losses.BinaryFocalCrossentropy(name = 'loss'),
    metrics=[tf.keras.metrics.CategoricalAccuracy(name = 'acc'), tf.keras.metrics.AUC(name = 'auc')],
)

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_auc',min_delta=0.001,patience=5,mode='max'),
]

model.fit(
    trainset.batch(256).prefetch(tf.data.experimental.AUTOTUNE), 
    validation_data=valset.batch(256).prefetch(tf.data.experimental.AUTOTUNE), 
    epochs =100, 
    callbacks=callbacks,
)

Epoch 1/100
1507/1507 [==============================] - 259s 159ms/step - loss: 0.0396 - acc: 0.4309 - auc: 0.8234 - val_loss: 0.0391 - val_acc: 0.3223 - val_auc: 0.7973
Epoch 2/100
1507/1507 [==============================] - 238s 158ms/step - loss: 0.0350 - acc: 0.4891 - auc: 0.8605 - val_loss: 0.0394 - val_acc: 0.2989 - val_auc: 0.7864
Epoch 3/100
1507/1507 [==============================] - 237s 157ms/step - loss: 0.0340 - acc: 0.5183 - auc: 0.8770 - val_loss: 0.0400 - val_acc: 0.3016 - val_auc: 0.7846
Epoch 4/100
1507/1507 [==============================] - 237s 157ms/step - loss: 0.0330 - acc: 0.5682 - auc: 0.8928 - val_loss: 0.0365 - val_acc: 0.4732 - val_auc: 0.8597
Epoch 5/100
1507/1507 [==============================] - 236s 157ms/step - loss: 0.0311 - acc: 0.6225 - auc: 0.9153 - val_loss: 0.0388 - val_acc: 0.4442 - val_auc: 0.7968
Epoch 6/100
1507/1507 [==============================] - 237s 157ms/step - loss: 0.0300 - acc: 0.6471 - auc: 0.9237 - val_loss: 0.0347 - val_acc:

In [11]:
import pendulum
ts = pendulum.now().int_timestamp

In [12]:
signature_dict = {
    inp.name : inp.type_spec
    for inp in inputs
}

pchain = ['0.99', '4.99', '9.99', '14.99', '19.99', '29.99', '49.99', '99.99']
price_chain_tensor = tf.constant(pchain, dtype=tf.string)

@tf.function(input_signature=[signature_dict])
def serve(x):
    pred = model(x)
    pred = tf.math.argmax(pred, axis = 1)
    pred = tf.gather(price_chain_tensor, pred)
    return pred

model.serve = serve
model.save_weights(f'../../savedmodel/weights/dynamic_pricing/0.0.2/{ts}')
model.save(f'../../savedmodel/dynamic_pricing/0.0.2/{ts}', signatures=model.serve)


INFO:tensorflow:Assets written to: ../../savedmodel/dynamic_pricing/0.0.2/1683870961\assets


INFO:tensorflow:Assets written to: ../../savedmodel/dynamic_pricing/0.0.2/1683870961\assets


In [10]:
weights = tf.softmax(price_chain_tensor) * 0.2 + 1

@tf.function(input_signature=[signature_dict])
def weighted_serve(x):
    pred = model(x)
    pred = tf.math.argmax(pred, axis = 1)
    pred = tf.gather(price_chain_tensor, pred)
    return pred


import pendulum
ts = pendulum.now().int_timestamp()
model.serve = serve
model.save_weights('../../savedmodel/weights/dynamic_pricing/0.0.2/12')
model.save(f'../../savedmodel/dynamic_pricing/0.0.2/{ts}', signatures=model.serve)

AttributeError: module 'tensorflow' has no attribute 'softmax'

In [13]:
# client = SQLClient('mix')

# client.execute_trino_query("select * from ml_tmp_db.training_sample_for_wgame_bundle_price_002 where grp = 'val_order' and dt >= '2023-04-27' ")
# test = client.to_pandas()
# test.to_parquet('../../data/raw/0.0.2.maxp7day.valid.pqt', compression='gzip')
test= pd.read_parquet('../../data/raw/0.0.2.maxp7day.valid.pqt')

In [15]:

test = test.assign(
    feature = lambda x: x.feature.apply(lambda x: fe.extract(x) if x is not None else fe.extract('')),
    bundle_price = lambda x: x.bundle_price.astype(np.int64),
    sid = (test['acc_id'] + test['role_id'] + test['bundle_price'].astype(str)).apply(CityHash32),
)

test = test.assign(
    ** {
        x : test.feature.apply(lambda y: y.get(f'r_{x}')[0])
        for x in feature_name
    }
).filter(items = feature_name + ['trigger_type','bundle_price','if_buy_pop_gift','sid'])


def gettestset():
    def get_label_func(x):
        price = x.pop('bundle_price')
        price = tf.one_hot(price, len(price_chain))
        label = x.pop('if_buy_pop_gift')
        return x, price, label
    
    tensors = tuple()
    for f in names:
        if f == 'bundle_price':
            v = test[f].apply(p2idx.get).values
        else:
            v = test[f].values
        t = tf.data.Dataset.from_tensor_slices(v, name = f)
        tensors = tensors + (t,)
        
    return tf.data.Dataset.zip(tensors).map(lambda *x: dict(zip(names, x))).map(get_label_func)


C:\Users\caoxu\AppData\Local\Temp\ipykernel_16224\4025256982.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test = test.assign(
C:\Users\caoxu\AppData\Local\Temp\ipykernel_16224\4025256982.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test = test.assign(


In [16]:
pchain = price_chain
price_chain_tensor = tf.constant(pchain, dtype=tf.float32) / 100

def serve(x):
    pred = model(x)
    pred = tf.math.argmax(pred, axis = 1)
    pred = tf.gather(price_chain_tensor, pred)
    return pred


def get_ture_price(y):
    y = tf.gather(price_chain_tensor, tf.math.argmax(y, axis = 1))
    y = y / 100
    return y

def bundle_price_evaluate(dataset):
    pred, price, label = [], [], []
    for x, p, y in dataset:
        pred.append(serve(x))
        price.append(get_ture_price(p))
        label.append(y)

    pred = tf.concat(pred, axis = 0)
    price = tf.concat(price, axis = 0)
    label = tf.concat(label, axis = 0)
    return pred, price, label

testset = gettestset()
pred, price, label = bundle_price_evaluate(testset.batch(256).prefetch(tf.data.experimental.AUTOTUNE))

In [22]:
badcase_rate = tf.logical_and(pred < price * 100, label > 0).numpy().sum() / (label > 0).numpy().sum()

badcase_loss = tf.cast(tf.logical_and(pred < price * 100, label > 0), tf.double) * tf.cast(price * 100 - pred, tf.double) 
badcase_loss = badcase_loss.numpy().sum() 


'{:.2%}'.format(badcase_rate), '{:.2}'.format(badcase_loss)

('3.67%', '6.2e+04')

In [23]:
greycase_rate = tf.logical_and(pred >  price * 100, label == 0).numpy().sum() / (label == 0).numpy().sum()

'{:.2%}'.format(greycase_rate)

'36.91%'

In [24]:
y,idx,cnt = tf.unique_with_counts(pred)
dist_normal = dict(zip(y.numpy(),cnt.numpy()/cnt.numpy().sum()))
dist_normal

{4.99: 0.3658323426484571,
 19.99: 0.38859598775724236,
 9.99: 0.14334910578035184,
 99.99: 0.05753079594040271,
 29.99: 0.03401344521564286,
 49.99: 0.010592786987669228,
 14.99: 7.933743326048339e-05,
 0.99: 6.198236973475265e-06}